In [64]:
import pandas as pd
import re
import numpy as np
import re
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from gensim.parsing.preprocessing import remove_stopwords




In [65]:
# Setting up model

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
df= pd.read_csv("data/songs_lyrics_filtered1959.csv")

In [67]:
def clean_text(text):

    text = text.replace('\n', ' ')
    text = re.sub(r'[,\.!?]', '', text)
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'\w*\d\w*',' ', text)
    text = re.sub(r'[()]', ' ', text)
    text = text.lower()
    text = re.sub(r'\b(chorus|verse|intro)\b', '', text)
    return text

In [68]:
df['lyrics_clean'] = df['lyrics'].astype(str).apply(lambda x: clean_text(x))
df

,Unnamed: 0,song_id,artist,title,tag,year,lyrics,lyrics_clean
0,0,lil_wayne_mr_carter,Lil Wayne,Mr. Carter,rap,2008,[Produced by Infamous and Drew Correa]\n\n[Int...,yo yo drew and inf this-this this right h...
1,1,birdman_pop_bottles,Birdman,Pop Bottles,rap,2007,"[Hook: Jadakiss, Lil Wayne & Birdman]\nStart w...",start with straight shots and then pop bottl...
2,2,lil_wayne_fireman,Lil Wayne,Fireman,rap,2005,"[Intro]\n(Weezy Baby)\nShh, the fireman comin'...",weezy baby shh the fireman comin' yeah yea...
3,3,ol_dirty_bastard_brooklyn_zoo,Ol' Dirty Bastard,Brooklyn Zoo,rap,1995,[Produced by True Master & Ol' Dirty Bastard]\...,shit word i'll bust that nigga ass right ...
4,4,lil_wayne_a_milli,Lil Wayne,A Milli,rap,2008,[Intro]\nBangladesh\nYoung Money!\nYou dig?\nM...,bangladesh young money you dig mack i'm goin...
...,...,...,...,...,...,...,...,...
20514,20544,jack_harlow_first_class,Jack Harlow,First Class,rap,2022,[Chorus: Jack Harlow & Fergie]\nMm\nI been a G...,mm i been a g throw up the l sex in the am u...
20515,20545,mary_macgregor_for_a_while,Mary Macgregor,For a While,pop,1976,[Verse 1]\nI think I'll stay around here for a...,i think i'll stay around here for a while i ...
20516,20546,mary_macgregor_dancin_like_lovers,Mary Macgregor,Dancin Like Lovers,pop,1980,[Verse 1]\nThe music's playing softly in the s...,the music's playing softly in the summer nig...
20517,20547,42_dugg_thump_shit,42 Dugg & EST Gee,Thump Shit,rap,2022,"[Intro: 42 Dugg]\nFree them boys, we them (Me ...",free them boys we them me and spiff ayy fr...


In [69]:
# Removing stopwords

df['lyrics_clean'] = df['lyrics_clean'].astype(str).apply(lambda x: remove_stopwords(x))

In [48]:
df_test = df.sample(n=100, random_state=42)

In [73]:
positive_scores = []
neutral_scores = []
negative_scores = []

for index, row in df.iterrows():
    lyrics = row['lyrics_clean']
    
    encoded_input = tokenizer(lyrics, return_tensors='pt', max_length=512, truncation=True, padding=True)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    
    # Initialize scores for each label
    positive_score = 0.0
    neutral_score = 0.0
    negative_score = 0.0
    
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        if l == 'positive':
            positive_score = np.round(float(s), 4)
        elif l == 'neutral':
            neutral_score = np.round(float(s), 4)
        elif l == 'negative':
            negative_score = np.round(float(s), 4)
    
    # Append scores to respective lists
    positive_scores.append(positive_score)
    neutral_scores.append(neutral_score)
    negative_scores.append(negative_score)

# Add the score columns to the DataFrame
df['positive_score'] = positive_scores
df['neutral_score'] = neutral_scores
df['negative_score'] = negative_scores



In [86]:
df

,song_id,artist,title,tag,year,lyrics,lyrics_clean,positive_score,neutral_score,negative_score,is_positive,is_negative,is_neutral,is_slightly_positive,is_slightly_negative
0,lil_wayne_mr_carter,Lil Wayne,Mr. Carter,rap,2008,[Produced by Infamous and Drew Correa]\n\n[Int...,yo yo drew inf this-this right crazy yo feel b...,0.1740,0.6079,0.2180,0,0,1,0,0
1,birdman_pop_bottles,Birdman,Pop Bottles,rap,2007,"[Hook: Jadakiss, Lil Wayne & Birdman]\nStart w...",start straight shots pop bottles yeah flirt ho...,0.2564,0.6455,0.0981,0,0,1,0,0
2,lil_wayne_fireman,Lil Wayne,Fireman,rap,2005,"[Intro]\n(Weezy Baby)\nShh, the fireman comin'...",weezy baby shh fireman comin' yeah yeah yeah y...,0.3448,0.5926,0.0626,0,0,1,0,0
3,ol_dirty_bastard_brooklyn_zoo,Ol' Dirty Bastard,Brooklyn Zoo,rap,1995,[Produced by True Master & Ol' Dirty Bastard]\...,shit word i'll bust nigga ass right ain't no– ...,0.0211,0.1343,0.8446,0,1,0,0,0
4,lil_wayne_a_milli,Lil Wayne,A Milli,rap,2008,[Intro]\nBangladesh\nYoung Money!\nYou dig?\nM...,bangladesh young money dig mack i'm going mill...,0.0406,0.2870,0.6725,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20514,jack_harlow_first_class,Jack Harlow,First Class,rap,2022,[Chorus: Jack Harlow & Fergie]\nMm\nI been a G...,mm g throw l sex uh-huh o-r-o-u-s yeah class s...,0.3042,0.6257,0.0701,0,0,1,0,0
20515,mary_macgregor_for_a_while,Mary Macgregor,For a While,pop,1976,[Verse 1]\nI think I'll stay around here for a...,think i'll stay like way talking smile think i...,0.7509,0.2295,0.0196,1,0,0,0,0
20516,mary_macgregor_dancin_like_lovers,Mary Macgregor,Dancin Like Lovers,pop,1980,[Verse 1]\nThe music's playing softly in the s...,music's playing softly summer night starlight'...,0.6386,0.3524,0.0091,1,0,0,0,0
20517,42_dugg_thump_shit,42 Dugg & EST Gee,Thump Shit,rap,2022,"[Intro: 42 Dugg]\nFree them boys, we them (Me ...",free boys spiff ayy free boys hip yeah ayy fre...,0.1053,0.3911,0.5037,0,1,0,0,0


In [75]:
positive_mask = (df['positive_score'] > df['negative_score']) & (df['positive_score'] > df['neutral_score'])
negative_mask = (df['negative_score'] > df['positive_score']) & (df['negative_score'] > df['neutral_score'])
neutral_mask = ~positive_mask & ~negative_mask

# Assign sentiment labels based on masks
df['is_positive'] = np.where(positive_mask, 1, 0)
df['is_negative'] = np.where(negative_mask, 1, 0)
df['is_neutral'] = np.where(neutral_mask, 1, 0)

In [93]:
df['is_positive'] = ((df['positive_score'] > df['neutral_score']) & (df['positive_score'] > df['negative_score'])).astype(int)
df['is_negative'] = ((df['negative_score'] > df['neutral_score']) & (df['negative_score'] > df['positive_score'])).astype(int)
df['is_slightly_positive'] = ((df['positive_score'] < df['neutral_score']) & (df['neutral_score'] > df['negative_score']) &(df['positive_score'] > df['negative_score']) ).astype(int)
df['is_slightly_negative'] = ((df['positive_score'] < df['neutral_score']) & (df['neutral_score'] > df['negative_score']) &(df['positive_score'] < df['negative_score']) ).astype(int)

In [94]:
df['is_slightly_negative'].value_counts()

is_slightly_negative
0    15790
1     4729
Name: count, dtype: int64

In [95]:
df.columns

Index(['song_id', 'artist', 'title', 'tag', 'year', 'lyrics', 'lyrics_clean',
       'positive_score', 'neutral_score', 'negative_score', 'is_positive',
       'is_negative', 'is_slightly_positive', 'is_slightly_negative'],
      dtype='object')

In [98]:
def determine_sentiment_label(row):
    if row['is_positive'] == 1:
        return 'positive'
    elif row['is_negative'] == 1:
        return 'negative'
    elif row['is_slightly_positive'] == 1:
        return 'slightly_positive'
    elif row['is_slightly_negative'] == 1:
        return 'slightly_negative'
    else:
        return 'neutral'

df['sentiment_label'] = df.apply(determine_sentiment_label, axis=1)

In [99]:
df

,song_id,artist,title,tag,year,lyrics,lyrics_clean,positive_score,neutral_score,negative_score,is_positive,is_negative,is_slightly_positive,is_slightly_negative,sentiment_label
0,lil_wayne_mr_carter,Lil Wayne,Mr. Carter,rap,2008,[Produced by Infamous and Drew Correa]\n\n[Int...,yo yo drew inf this-this right crazy yo feel b...,0.1740,0.6079,0.2180,0,0,0,1,slightly_negative
1,birdman_pop_bottles,Birdman,Pop Bottles,rap,2007,"[Hook: Jadakiss, Lil Wayne & Birdman]\nStart w...",start straight shots pop bottles yeah flirt ho...,0.2564,0.6455,0.0981,0,0,1,0,slightly_positive
2,lil_wayne_fireman,Lil Wayne,Fireman,rap,2005,"[Intro]\n(Weezy Baby)\nShh, the fireman comin'...",weezy baby shh fireman comin' yeah yeah yeah y...,0.3448,0.5926,0.0626,0,0,1,0,slightly_positive
3,ol_dirty_bastard_brooklyn_zoo,Ol' Dirty Bastard,Brooklyn Zoo,rap,1995,[Produced by True Master & Ol' Dirty Bastard]\...,shit word i'll bust nigga ass right ain't no– ...,0.0211,0.1343,0.8446,0,1,0,0,negative
4,lil_wayne_a_milli,Lil Wayne,A Milli,rap,2008,[Intro]\nBangladesh\nYoung Money!\nYou dig?\nM...,bangladesh young money dig mack i'm going mill...,0.0406,0.2870,0.6725,0,1,0,0,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20514,jack_harlow_first_class,Jack Harlow,First Class,rap,2022,[Chorus: Jack Harlow & Fergie]\nMm\nI been a G...,mm g throw l sex uh-huh o-r-o-u-s yeah class s...,0.3042,0.6257,0.0701,0,0,1,0,slightly_positive
20515,mary_macgregor_for_a_while,Mary Macgregor,For a While,pop,1976,[Verse 1]\nI think I'll stay around here for a...,think i'll stay like way talking smile think i...,0.7509,0.2295,0.0196,1,0,0,0,positive
20516,mary_macgregor_dancin_like_lovers,Mary Macgregor,Dancin Like Lovers,pop,1980,[Verse 1]\nThe music's playing softly in the s...,music's playing softly summer night starlight'...,0.6386,0.3524,0.0091,1,0,0,0,positive
20517,42_dugg_thump_shit,42 Dugg & EST Gee,Thump Shit,rap,2022,"[Intro: 42 Dugg]\nFree them boys, we them (Me ...",free boys spiff ayy free boys hip yeah ayy fre...,0.1053,0.3911,0.5037,0,1,0,0,negative


In [100]:
df.to_csv("sentiment_save.csv")